In [78]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import datetime
from datetime import time
import numpy as np

def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape

# set up the interval need to forecast here
unique_times = [ datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30)
]

#for month_number in ['2024-11-15']:
for month_number in ['2024-11-15']:
    #ETL Process
    data = pd.read_excel('CS South WFM STAT 2025_05_14.xlsx')
    pd.set_option('display.max_rows', 100)
    data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
           'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
           'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']
    
    data = data[~data .offered.isna()]
    
    data = data[data .offered != 0]
    # Month one and half need to set up 
    data_=data
    data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
    holiday_data = pd.read_excel(r'holiday.xlsx')
    holiday_data['~is_holiday'] = 0
    Holiday_name = ['Christmas Day', 'Columbus Day',
           'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
           'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
           'Veterans Day']
    holiday_data['date'] = holiday_data.Date.dt.date
    data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
    data['date'] = data.datetime.dt.date
    
    data.datetime = data.datetime.dt.floor('T')
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('30T') if x.minute == 29 else x
    )
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('H') if x.minute == 59 else x
    )
    data['time'] = data.datetime.dt.time
    data = data.sort_values('datetime').reset_index(drop = True)
    data = data.sort_values(by='datetime')
    
    # 获取所有唯一的时间点（如每天的09:00, 09:30等）
    unique_times = data['time'].unique()
    
    # 获取所有唯一的日期
    unique_dates = data['date'].unique()
    data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
    data = data[data['~is_holiday'] == 1]
    
    holiday_datetime = holiday_data.date.to_numpy()
    
    start_time = 35+23 # 58 how many days in advance
    end_time = start_time+36 #36 is the train length

    
    add_column = ['actans','actabn','absActHt','absActSa']
    
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    
    matrix = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
        
        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix)
    
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)
    
    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
    
    matrix = matrix.sort_values('datetime').reset_index(drop = True)
    
    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)
    
    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))
    
    
    
    fill_na = fill_na.interpolate(method='linear')
    
    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)
    
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)
    
    matrix_predict = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()    
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']  
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-start_time:]
        
        
        
        # 将结果存入矩阵
        matrix_predict.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix_predict)
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix_predict)):
        if matrix_predict[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix_predict[i] = np.vstack([matrix_predict[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix_predict = np.vstack(matrix_predict)
    
    matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
    matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
    matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
    matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
    matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)
    
    matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)
    
    read_data_predict = matrix_predict.iloc[:]
    
    
    
    
    fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))
    
    fill_na_predict = fill_na_predict.interpolate(method='linear')
    fill_na_predict = fill_na_predict.fillna(1)
    
    fill_na_predict['datetime'] = read_data_predict.datetime
    fill_na_predict['date'] = fill_na_predict.datetime.dt.date
    
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
    fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
    fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
    
    fill_na_predict
    
    X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns
                
    X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns
    
    
    lstm_data = fill_na.dropna(axis = 0)
    
    lstm_data
    
    for i in X_columns:
        try:
            fill_na_predict[i]
        except:
            fill_na_predict[i] = 0
    
    lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
    
    week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
    
    lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
    fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])
    all_feature = pd.DataFrame()
    all_predict=pd.DataFrame()
    #for week_ in [1,2,3,4,5]:
    for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
        now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
        #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]
    
        now_time_list = now_lstm_data.datetime.to_numpy()
        now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
        # 归一化数据
    
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data
    
        X_test_1, y_test =now_lstm_data[X_columns][-start_time:] , now_lstm_data[['offered']][-start_time:]
    
        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(LSTM(50, return_sequences=False))
        model_1.add(Dense(25))
        model_1.add(Dense(1))
    
        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler.inverse_transform(train_predict_1)
        True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-start_time:]
        now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
    
        X_train_2, y_train =now_lstm_data[X_columns2][:-start_time] , scaled_data
    
        X_test_2, y_test =now_lstm_data[X_columns2][-start_time:] , now_lstm_data[['offered']][-start_time:]
    
        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))
    
        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler.inverse_transform(train_predict_2)
        True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-start_time:]
        print(calculate_mape(y_test, return_y_pred_2))
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):
            all_predict = pd.concat([now_predict_1,all_predict])
            future_X = now_fill_na_predict[X_columns].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        else:
            all_predict = pd.concat([now_predict_2,all_predict])
            future_X = now_fill_na_predict[X_columns2].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        #print('Interval Hour:{}, Minute:{} has been trained'.format(hour,minute)
    all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
    #all_feature.to_excel(r'return_month_{}.xlsx'.format(month_number))
    all_predict['diff'] = (all_predict.predict - all_predict.offered)
    all_predict['hour'] = all_predict.datetime.dt.hour
    pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    calculate_mape(all_predict.offered,all_predict.predict)
    all_predict.to_excel(r'Test_{}.xlsx'.format(month_number))
    time_ = []
    for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        time_.append(now_fill_na_predict.datetime.to_numpy())
    all_feature['datetime'] =  np.hstack(time_)
    all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
    data_['conversation_start_interval_tmst']=pd.to_datetime(data_['conversation_start_interval_tmst'])
    all_feature['datetime']=pd.to_datetime(all_feature['datetime'])
    all_feature = pd.merge(all_feature,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])
    all_feature = all_feature.sort_values('datetime').reset_index(drop = True)
    all_feature['mape']=np.abs(all_feature['offered']-all_feature['predict'])/(all_feature['offered'])*100
    all_feature.to_excel(r'Target_{}.xlsx'.format(month_number))

/tmp/ipykernel_272967/127859848.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
/tmp/ipykernel_272967/127859848.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data.datetime.dt.date
/tmp/ipykernel_272967/127859848.py:76: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')
/tmp/ipykernel_272967/127859848.py:76: SettingW

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step - loss: 0.0737
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step - loss: 0.0557
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step - loss: 0.0440
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - loss: 0.0484
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - loss: 0.0319
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - loss: 0.0510
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step - loss: 0.0427
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - loss: 0.0421
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step - loss: 0.0582
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - loss: 0.0372
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step - loss: 0.0313
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step - loss: 0.0404
Epoch 13/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - loss: 0.0345
Epoch 14/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step - loss: 0.0363
Epoch 15/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step - loss: 0.0383
Epoch 16/20
3/3 ━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.5484
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - loss: 0.0982
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - loss: 0.0023
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 0.0860
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - loss: 0.1480
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - loss: 0.1366
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - loss: 0.0885
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - loss: 0.0403
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.0098
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - loss: 2.0604e-05
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - loss: 0.0055
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - loss: 0.0180
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - loss: 0.0298
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - loss: 0.0366
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 0.0367
Epoch 16/20
1/1 ━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.1355
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 0.0233
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - loss: 0.0014
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - loss: 0.0277
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.0421
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - loss: 0.0346
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - loss: 0.0191
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - loss: 0.0063
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step - loss: 4.6268e-04
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - loss: 8.4565e-04
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - loss: 0.0046
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - loss: 0.0087
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - loss: 0.0109
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - loss: 0.0107
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - loss: 0.0085
Epoch 16/20
1/1 ━━━━━━━━━

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [89]:
now_lstm_data

,offered,datetime,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,...,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94
0,1.0,2023-01-23 07:00:00,4.333333,5.333333,2.00,28.333333,2.8,2.166667,1.2,1.00,...,44738.383394,8310.552381,22.000000,1.666667,17025.000000,2624.266667,1.0,0.2,0.000000,1.600000
1,1.0,2024-10-04 07:00:00,23.000000,27.500000,25.00,13.000000,26.0,19.500000,10.0,19.00,...,10485.333333,271.555556,13.666667,0.000000,40230.712121,185.270833,22.5,0.0,51051.282353,570.901357
2,1.0,2024-10-28 07:00:00,10.250000,11.250000,17.25,15.500000,23.0,12.750000,10.5,12.25,...,15113.158333,215.691667,8.500000,0.000000,20154.841667,192.650000,13.0,0.0,41623.854167,317.355921


In [83]:
lstm_data

,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,...,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,week_num
0,1.0,2.0,0.0,743.000000,29.000000,1.0,0.0,241.000000,7.000000,2.0,...,0,0,1,0,0,1,0,0,0,1
1,2.0,5.0,0.0,0.000000,2725.000000,3.0,0.0,0.000000,39.000000,2.0,...,0,0,1,0,0,1,0,0,0,1
2,8.0,9.0,0.0,13099.500000,3399.300000,5.0,1.0,5437.500000,1836.666667,2.0,...,0,0,1,0,0,1,0,0,0,1
3,9.0,12.0,0.0,41430.000000,1380.000000,10.0,0.0,17918.000000,103.809524,9.0,...,0,0,1,0,0,1,0,0,0,1
4,24.0,26.0,2.0,48023.772727,6383.333333,23.0,2.0,35699.513889,4818.918182,26.0,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16681,18.0,13.0,1.0,7203.256000,328.000000,14.0,2.0,6953.976000,708.000000,9.0,...,0,0,0,0,0,0,1,0,0,4
16682,2.0,13.0,1.0,7203.256000,328.000000,0.0,1.0,1.000000,1.000000,0.0,...,0,0,0,0,0,0,1,0,0,4
16683,1.0,13.0,1.0,7203.256000,328.000000,0.0,1.0,1.000000,1.000000,0.0,...,0,0,0,0,0,0,1,0,0,4
16684,1.0,13.0,1.0,7203.256000,328.000000,0.0,1.0,1.000000,1.000000,0.0,...,0,0,0,0,0,0,1,0,0,4


In [82]:
start_time

58

In [86]:
#RS KSR Script

import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import datetime
from datetime import time
import numpy as np

def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape

unique_times = [ 
datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0),
datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),datetime.time(18,0),datetime.time(18,30),datetime.time(19,0)
]

#for month_number in ['2024-11-15']:
for month_number in ['2024-11-15','2024-12-15','2025-1-15','2025-2-15','2025-3-15']:
    #ETL Process
    data = pd.read_excel('RS_KSR_WFM_STAT_2025_05_14.xlsx')
    pd.set_option('display.max_rows', 100)
    data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
           'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
           'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']
    
    data = data[~data .offered.isna()]
    
    data = data[data .offered != 0]
    # Month one and half need to set up 
    data_=data
    data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
    holiday_data = pd.read_excel(r'holiday.xlsx')
    holiday_data['~is_holiday'] = 0
    Holiday_name = ['Christmas Day', 'Columbus Day',
           'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
           'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
           'Veterans Day']
    holiday_data['date'] = holiday_data.Date.dt.date
    data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
    data['date'] = data.datetime.dt.date
    
    data.datetime = data.datetime.dt.floor('T')
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('30T') if x.minute == 29 else x
    )
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('H') if x.minute == 59 else x
    )
    data['time'] = data.datetime.dt.time
    data = data.sort_values('datetime').reset_index(drop = True)
    data = data.sort_values(by='datetime')
    
    # 获取所有唯一的时间点（如每天的09:00, 09:30等）
    unique_times = data['time'].unique()
    
    # 获取所有唯一的日期
    unique_dates = data['date'].unique()
    data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
    data = data[data['~is_holiday'] == 1]
    
    holiday_datetime = holiday_data.date.to_numpy()
    
    start_time = 35+23 # 58 how many days in advance
    end_time = start_time+36 #36 is the train length
    # set up the interval need to forecast here

    
    add_column = ['actans','actabn','absActHt','absActSa']
    
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    
    matrix = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
        
        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix)
    
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)
    
    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
    
    matrix = matrix.sort_values('datetime').reset_index(drop = True)
    
    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)
    
    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))
    
    
    
    fill_na = fill_na.interpolate(method='linear')
    
    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)
    
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)
    
    matrix_predict = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()    
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']  
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-start_time:]
        
        
        
        # 将结果存入矩阵
        matrix_predict.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix_predict)
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix_predict)):
        if matrix_predict[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix_predict[i] = np.vstack([matrix_predict[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix_predict = np.vstack(matrix_predict)
    
    matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
    matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
    matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
    matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
    matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)
    
    matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)
    
    read_data_predict = matrix_predict.iloc[:]
    
    
    
    
    fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))
    
    fill_na_predict = fill_na_predict.interpolate(method='linear')
    fill_na_predict = fill_na_predict.fillna(1)
    
    fill_na_predict['datetime'] = read_data_predict.datetime
    fill_na_predict['date'] = fill_na_predict.datetime.dt.date
    
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
    fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
    fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
    
    fill_na_predict
    
    X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns
                
    X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns
    
    
    lstm_data = fill_na.dropna(axis = 0)
    
    lstm_data
    
    for i in X_columns:
        try:
            fill_na_predict[i]
        except:
            fill_na_predict[i] = 0
    
    lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
    
    week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
    
    lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
    fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])
    all_feature = pd.DataFrame()
    all_predict=pd.DataFrame()
    #for week_ in [1,2,3,4,5]:
    for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
        now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
        #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]
    
        now_time_list = now_lstm_data.datetime.to_numpy()
        now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
        # 归一化数据
    
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data
    
        X_test_1, y_test =now_lstm_data[X_columns][-start_time:] , now_lstm_data[['offered']][-start_time:]
    
        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(LSTM(50, return_sequences=False))
        model_1.add(Dense(25))
        model_1.add(Dense(1))
    
        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler.inverse_transform(train_predict_1)
        True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-start_time:]
        now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
    
        X_train_2, y_train =now_lstm_data[X_columns2][:-start_time] , scaled_data
    
        X_test_2, y_test =now_lstm_data[X_columns2][-start_time:] , now_lstm_data[['offered']][-start_time:]
    
        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))
    
        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler.inverse_transform(train_predict_2)
        True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-start_time:]
        print(calculate_mape(y_test, return_y_pred_2))
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):
            all_predict = pd.concat([now_predict_1,all_predict])
            future_X = now_fill_na_predict[X_columns].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        else:
            all_predict = pd.concat([now_predict_2,all_predict])
            future_X = now_fill_na_predict[X_columns2].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        #print('Interval Hour:{}, Minute:{} has been trained'.format(hour,minute)
    all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
    #all_feature.to_excel(r'return_month_{}.xlsx'.format(month_number))
    all_predict['diff'] = (all_predict.predict - all_predict.offered)
    all_predict['hour'] = all_predict.datetime.dt.hour
    pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    calculate_mape(all_predict.offered,all_predict.predict)
    all_predict.to_excel(r'RS_KSR_Test_{}.xlsx'.format(month_number))
    time_ = []
    for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        time_.append(now_fill_na_predict.datetime.to_numpy())
    all_feature['datetime'] =  np.hstack(time_)
    all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
    data_['conversation_start_interval_tmst']=pd.to_datetime(data_['conversation_start_interval_tmst'])
    all_feature['datetime']=pd.to_datetime(all_feature['datetime'])
    all_feature = pd.merge(all_feature,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])
    all_feature = all_feature.sort_values('datetime').reset_index(drop = True)
    all_feature['mape']=np.abs(all_feature['offered']-all_feature['predict'])/(all_feature['offered'])*100
    all_feature.to_excel(r'RS_KSR_Target_{}.xlsx'.format(month_number))

/tmp/ipykernel_272967/512764376.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
/tmp/ipykernel_272967/512764376.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data.datetime.dt.date
/tmp/ipykernel_272967/512764376.py:78: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')
/tmp/ipykernel_272967/512764376.py:78: SettingW

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [87]:
now_lstm_data

,offered,datetime,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,...,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94
0,1.0,2023-01-23 07:00:00,4.333333,5.333333,2.00,28.333333,2.8,2.166667,1.2,1.00,...,44738.383394,8310.552381,22.000000,1.666667,17025.000000,2624.266667,1.0,0.2,0.000000,1.600000
1,1.0,2024-10-04 07:00:00,23.000000,27.500000,25.00,13.000000,26.0,19.500000,10.0,19.00,...,10485.333333,271.555556,13.666667,0.000000,40230.712121,185.270833,22.5,0.0,51051.282353,570.901357
2,1.0,2024-10-28 07:00:00,10.250000,11.250000,17.25,15.500000,23.0,12.750000,10.5,12.25,...,15113.158333,215.691667,8.500000,0.000000,20154.841667,192.650000,13.0,0.0,41623.854167,317.355921


In [ ]:

all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
#all_feature.to_excel(r'return_month_{}.xlsx'.format(month_number))
all_predict['diff'] = (all_predict.predict - all_predict.offered)
all_predict['hour'] = all_predict.datetime.dt.hour
pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
calculate_mape(all_predict.offered,all_predict.predict)
all_predict.to_excel(r'Test_{}.xlsx'.format(month_number))
time_ = []
for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    time_.append(now_fill_na_predict.datetime.to_numpy())
all_feature['datetime'] =  np.hstack(time_)
all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
data_['conversation_start_interval_tmst']=pd.to_datetime(data_['conversation_start_interval_tmst'])
all_feature['datetime']=pd.to_datetime(all_feature['datetime'])
all_feature = pd.merge(all_feature,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])
all_feature = all_feature.sort_values('datetime').reset_index(drop = True)
all_feature['mape']=np.abs(all_feature['offered']-all_feature['predict'])/(all_feature['offered'])*100
all_feature.to_excel(r'Target_{}.xlsx'.format(month_number))

In [71]:
time_ = []
for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    time_.append(now_fill_na_predict.datetime.to_numpy())
all_feature['datetime'] =  np.hstack(time_)
all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
data_['conversation_start_interval_tmst']=pd.to_datetime(data_['conversation_start_interval_tmst'])
all_feature['datetime']=pd.to_datetime(all_feature['datetime'])
all_feature = pd.merge(all_feature,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])
all_feature = all_feature.sort_values('datetime').reset_index(drop = True)
all_feature['mape']=np.abs(all_feature['offered']-all_feature['predict'])/(all_feature['offered'])*100
all_feature.to_excel(r'Target_{}.xlsx'.format(month_number))

In [88]:
lstm_data

,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,...,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,week_num
0,37.0,32.0,1.0,59345.142857,10578.598291,29.0,2.0,42378.666667,18984.573810,35.0,...,0,0,1,0,1,0,0,0,0,5
1,17.0,34.0,3.0,67703.807692,42008.133333,24.0,3.0,42739.285714,18532.800000,24.0,...,0,0,1,0,1,0,0,0,0,5
2,33.0,24.0,17.0,79839.384615,67299.619048,34.0,3.0,119072.250000,46178.314286,26.0,...,0,0,1,0,1,0,0,0,0,5
3,40.0,33.0,9.0,79216.400000,113523.850000,27.0,1.0,68695.071429,47624.700000,21.0,...,0,0,1,0,1,0,0,0,0,5
4,38.0,20.0,6.0,92581.000000,73993.000000,58.0,5.0,137439.120000,34600.866667,32.0,...,0,0,1,0,1,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10720,22.0,19.0,1.0,39421.079365,12697.911111,17.0,0.0,47067.414286,1606.095238,19.0,...,0,0,0,0,0,0,1,0,0,4
10721,18.0,11.0,3.0,33429.733333,11964.333333,21.0,5.0,80904.500000,27354.000000,31.0,...,0,0,0,0,0,0,1,0,0,4
10722,12.0,13.0,3.0,40212.250000,32933.952381,21.0,2.0,53200.250000,4747.166667,20.0,...,0,0,0,0,0,0,1,0,0,4
10723,9.0,13.0,3.0,67221.700000,16730.133333,11.0,0.0,19897.166667,221.466667,15.0,...,0,0,0,0,0,0,1,0,0,4


In [56]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mape ：',calculate_mape(_.predict, _.offered))

hour is 9.0 the mape ： 15.108565395142312
hour is 10.0 the mape ： 14.543283188637835
hour is 11.0 the mape ： 14.113005774019918
hour is 12.0 the mape ： 12.645508299088789
hour is 13.0 the mape ： 12.244234930406705
hour is 14.0 the mape ： 13.019868702897055
hour is 15.0 the mape ： 13.78376280752372
hour is 16.0 the mape ： 19.91938217071393
hour is 17.0 the mape ： 23.89058659019716
hour is nan the mape ： nan


/usr/local/lib/python3.10/dist-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mse ：',(np.mean((_.predict-_.offered)**2))**0.5)

In [ ]:
_ = all_predict[all_predict.hour == 11]

In [ ]:
calculate_mape(_.predict, _.offered)

In [54]:
all_feature_ = pd.merge(all_feature_,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])


In [52]:
all_feature_=all_feature[all_feature['datetime'].notna()]

In [67]:
(all_feature_[all_feature_['datetime']>=pd.to_datetime('2025-01-20 00:00:00')]['mape']).mean()

np.float64(25.134611845200947)

In [43]:
pd.set_option('display.max_rows', None)

In [65]:
all_feature_[all_feature_['datetime']>=pd.to_datetime('2025-01-10 00:00:00')]['mape']

738     33.208509
739      8.285545
740     10.427519
741     19.570874
742     15.077025
743     14.471300
744     16.976033
745      0.516770
746     25.660279
747     15.891084
748     17.562603
749     27.768948
750     23.178524
751      8.534840
752      6.079927
753     16.742529
754     21.701106
755      7.490306
756     30.779100
757     38.321014
758     29.270353
759     32.218009
760     32.037269
761     33.295054
762     34.275165
763     36.718296
764     38.162828
765     32.324275
766     41.023315
767     35.946070
768     33.119583
769     27.651849
770     25.984501
771     30.806241
772     45.436389
773     32.127987
774     41.158032
775     26.985486
776     27.486449
777     25.744703
778     27.339127
779     26.948297
780     32.965840
781     17.338157
782     41.068036
783     26.684955
784     37.725078
785     25.027156
786     26.288829
787     40.379822
788     26.710627
789     22.060587
790     54.258946
791     34.754110
792     33.948697
793     34

In [68]:
all_feature.to_excel(r'Target_{}.xlsx'.format('2025-01-01'))

In [ ]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)


In [ ]:
all_predict.to_excel(r'return.xlsx')

In [ ]:
datetime

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
data11 = lstm_data[lstm_data.hour == 11]

In [ ]:
data11.groupby('datetime').min().offered.sort_values()

In [ ]:

import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

# 创建示例数据
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['offered'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='真实值'  # 图例名称
))
fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['predict'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='预测值'  # 图例名称
))

# 设置坐标轴名称
fig.update_layout(
    title='折线图示例',
    xaxis_title='X轴',
    yaxis_title='Y轴',
    template='plotly_white'
)
fig.show()

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot( all_predict['datetime'], all_predict['offered'] , label='Actual Data')
plt.plot( all_predict['datetime'], all_predict['predict'], label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
now_lstm_data

In [ ]:
now_lstm_data.offered.min()

In [ ]:
_ = lstm_data.groupby(['year','month','day']).offered.sum()
_.sort_values()[:30]

In [ ]:
now_lstm_data[X_columns]

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot(now_lstm_data[['offered']][1000:].to_numpy(), label='Actual Data')
plt.plot(return_y_pred, label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
predict_y =  model.predict( now_lstm_data[X_columns][:1000].to_numpy().reshape(-1, X_train.shape[1], 1))
predict_y = scaler.inverse_transform(np.array(predict_y).reshape(-1, 1))

In [ ]:
calculate_mape(now_lstm_data[['offered']][:1000].to_numpy(), predict_y)

In [ ]:
_ = ((np.abs((now_lstm_data[['offered']][1000:].to_numpy() - return_y_pred)/now_lstm_data[['offered']][1000:].to_numpy())))*100

In [ ]:
_.sort()

In [ ]:
print

In [ ]:
np.argmax(_)

In [ ]:
_[298]

In [ ]:
now_lstm_data[['offered']][1000:].to_numpy()[298]

In [91]:
%config Completer.use_jedi = True

In [ ]:
import p